In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#import review corpus
y = pd.read_csv('revcorpus.csv')

#import word2vec model
embedding_vector_size = 300
wordvec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#import xpad
X_pad = np.load('xpad.npy',allow_pickle=False)

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 150
model = Sequential()

model.add(Embedding(input_dim=wordvec_model.wv.vectors.shape[0],
                    output_dim=wordvec_model.wv.vectors.shape[1],
                    weights=[wordvec_model.wv.vectors],
                    input_length=embed_size))

#model.add(Embedding(max_features,embed_size))

model.add(Bidirectional(CuDNNLSTM(64,return_sequences = True)))

model.add(GlobalMaxPool1D())

model.add(Dropout(0.1))
    
model.add(Dense(128,
                #activation=keras.layers.LeakyReLU(alpha=0.2),
                use_bias=True,
                kernel_initializer='lecun_uniform',
                bias_initializer='lecun_uniform',
                kernel_regularizer=keras.regularizers.l2(0.00),
                bias_regularizer=None,
                activity_regularizer=None,
                #kernel_constraint=keras.constraints.MaxNorm(2.),
                bias_constraint=None))

model.add(Activation(keras.layers.LeakyReLU(alpha=0.3)))

model.add(Dropout(0.00))

model.add(Dense(32, 
                #activation=keras.layers.LeakyReLU(alpha=0.2),
                use_bias=True,
                kernel_initializer='lecun_uniform',
                bias_initializer='lecun_uniform',
                kernel_regularizer=keras.regularizers.l2(0.00),
                bias_regularizer=None,
                activity_regularizer=None,
                #kernel_constraint=keras.constraints.MaxNorm(2.),
                bias_constraint=None))

model.add(Activation(keras.layers.LeakyReLU(alpha=0.2)))

model.add(Dropout(0.05))

model.add(Dense(1,
                activation='sigmoid',
                use_bias=True,
                kernel_initializer='lecun_uniform',
                bias_initializer='lecun_uniform',
                kernel_regularizer=keras.regularizers.l2(0.00),
                bias_regularizer=None,
                activity_regularizer=None,
                #kernel_constraint=keras.constraints.MaxNorm(2.),
                bias_constraint=None))

model.summary()

Using TensorFlow backend.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, shuffle=True)
X_test

In [ ]:
batch_size = 128
epochs = 20

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1)

# validate the model on test dataset to determine generalization
print("\n")
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))